In [ ]:
#!pip install pycryptodome
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import json
import random

class IoTDevice:
    def __init__(self, device_id):
        self.device_id = device_id
        self.key = get_random_bytes(16)  # AES-128 key
        self.iv = get_random_bytes(16)

    def generate_sensor_data(self):
        return {
            'device_id': self.device_id,
            'temperature': round(random.uniform(20.0, 35.0), 2),
            'humidity': round(random.uniform(30.0, 80.0), 2)
        }

    def encrypt_data(self, data):
        json_data = json.dumps(data)
        padded = json_data.ljust((len(json_data) // 16 + 1) * 16)
        cipher = AES.new(self.key, AES.MODE_CBC, self.iv)
        return cipher.encrypt(padded.encode())

    def get_key_iv(self):
        return self.key, self.iv


class Server:
    def decrypt_data(self, encrypted_data, key, iv):
        cipher = AES.new(key, AES.MODE_CBC, iv)
        decrypted = cipher.decrypt(encrypted_data)
        return json.loads(decrypted.decode().strip())


def main():

    # IoT Device Side
    device = IoTDevice("Sensor_001")

    # Generate sensor data
    data = device.generate_sensor_data()
    print(" Original Sensor Data:")
    print(f"   Temperature: {data['temperature']}°C")
    print(f"   Humidity: {data['humidity']}%\n")

    # Encrypt data
    encrypted = device.encrypt_data(data)
    print(" Encrypted Data (hex):")
    print(encrypted.hex()[:50] + "...\n")

    # Simulated Secure Transmission
    print(" Simulating Secure Transmission...")
    print("   Data sent securely to server \n")

    # Server Side
    server = Server()
    key, iv = device.get_key_iv()
    decrypted = server.decrypt_data(encrypted, key, iv)

    print("  Decrypted Sensor Data:")
    print(f"   Temperature: {decrypted['temperature']}°C")
    print(f"   Humidity: {decrypted['humidity']}%")

    # Verification
    print(f"\n Verification: Data integrity maintained = {data == decrypted}")

if __name__ == "__main__":
    main()